In [1]:
%matplotlib inline

import os
import shutil
import time

from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import scipy
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import DataLoader

In [51]:
# Dataset Parameters
batch_size = 200
load_size = 128
fine_size = 128
c = 3
data_mean = np.asarray([0.45834960097,0.44674252445,0.41352266842])

use_cuda = True

model_path = '../../data/models/resnet50_ep2.pt'

In [3]:
def construct_dataloader_test():
    # Construct DataLoader
    opt_data_test = {
        'data_root': '../../data/images/',
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean
    }
    
    return DataLoader.DataLoaderDiskTest(**opt_data_test)

In [4]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [14]:
def test(test_loader, model):
    # switch to evaluate mode
    model.eval()

    preds = []
    
    for i in range(test_loader.size() // batch_size):
        input, paths = test_loader.next_batch(batch_size)
        if use_cuda:
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)

        # compute output
        output = model(input_var)
        preds.append((paths, output))

        print('Test: [{0}/{1}]\t'.format((i+1)*batch_size, test_loader.size()))

    return preds

In [11]:
test_loader = construct_dataloader_test()

# Images found: 10000


In [12]:
model = torch.load(model_path)

In [17]:
model
preds = test(test_loader, model)

Test: [200/50.0]	
Test: [400/50.0]	
Test: [600/50.0]	
Test: [800/50.0]	
Test: [1000/50.0]	
Test: [1200/50.0]	
Test: [1400/50.0]	
Test: [1600/50.0]	
Test: [1800/50.0]	
Test: [2000/50.0]	
Test: [2200/50.0]	
Test: [2400/50.0]	
Test: [2600/50.0]	
Test: [2800/50.0]	
Test: [3000/50.0]	
Test: [3200/50.0]	
Test: [3400/50.0]	
Test: [3600/50.0]	
Test: [3800/50.0]	
Test: [4000/50.0]	
Test: [4200/50.0]	
Test: [4400/50.0]	
Test: [4600/50.0]	
Test: [4800/50.0]	
Test: [5000/50.0]	
Test: [5200/50.0]	
Test: [5400/50.0]	
Test: [5600/50.0]	
Test: [5800/50.0]	
Test: [6000/50.0]	
Test: [6200/50.0]	
Test: [6400/50.0]	
Test: [6600/50.0]	
Test: [6800/50.0]	
Test: [7000/50.0]	
Test: [7200/50.0]	
Test: [7400/50.0]	
Test: [7600/50.0]	
Test: [7800/50.0]	
Test: [8000/50.0]	
Test: [8200/50.0]	
Test: [8400/50.0]	
Test: [8600/50.0]	
Test: [8800/50.0]	
Test: [9000/50.0]	
Test: [9200/50.0]	
Test: [9400/50.0]	
Test: [9600/50.0]	
Test: [9800/50.0]	
Test: [10000/50.0]	


In [26]:
preds = preds[-2:] + preds[:-2]

In [44]:
probs = preds

In [64]:
probs[0]

(['../../data/images/test/00000001.jpg',
  '../../data/images/test/00000002.jpg',
  '../../data/images/test/00000003.jpg',
  '../../data/images/test/00000004.jpg',
  '../../data/images/test/00000005.jpg',
  '../../data/images/test/00000006.jpg',
  '../../data/images/test/00000007.jpg',
  '../../data/images/test/00000008.jpg',
  '../../data/images/test/00000009.jpg',
  '../../data/images/test/00000010.jpg',
  '../../data/images/test/00000011.jpg',
  '../../data/images/test/00000012.jpg',
  '../../data/images/test/00000013.jpg',
  '../../data/images/test/00000014.jpg',
  '../../data/images/test/00000015.jpg',
  '../../data/images/test/00000016.jpg',
  '../../data/images/test/00000017.jpg',
  '../../data/images/test/00000018.jpg',
  '../../data/images/test/00000019.jpg',
  '../../data/images/test/00000020.jpg',
  '../../data/images/test/00000021.jpg',
  '../../data/images/test/00000022.jpg',
  '../../data/images/test/00000023.jpg',
  '../../data/images/test/00000024.jpg',
  '../../data/im

In [62]:
with open(model_path + '_preds.txt', 'w') as f:
    for paths, batch_probs in preds:
        batch_preds = probs[0][1].data.topk(5, 1, True, True)[1]
#         batch_preds = batch_probs.data.numpy().argsort(axis=1)[:, :5]
        for path, top5 in zip(paths, batch_preds):
            path = 'test/' + os.path.basename(path)
            f.write('{} {} {} {} {} {}\n'.format(path, *top5))